In [1]:
#Import dependencies
import numpy as np
import pandas as pd
import tensorflow

In [2]:
#Import efficiecy stats
reg_season = pd.read_csv("efficiency_stats.csv")

#Import csvs that give the team's results and seeds for each year
tourney_result = pd.read_csv('WNCAATourneyCompactResults.csv')
tourney_seed = pd.read_csv('WNCAATourneySeeds.csv')

In [3]:
#Select only the winning and losing team ids for each season
tourney_result = tourney_result[['Season', 'WTeamID', 'LTeamID']]
tourney_result

,Season,WTeamID,LTeamID
0,1998,3104,3422
1,1998,3112,3365
2,1998,3163,3193
3,1998,3198,3266
4,1998,3203,3208
...,...,...,...
1381,2019,3124,3234
1382,2019,3323,3390
1383,2019,3124,3332
1384,2019,3323,3163


In [4]:
#Merge seed with winning team ids
result_seedW = pd.merge(tourney_result, tourney_seed, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])
result_seedW.rename(columns={'Seed': 'WSeed'}, inplace=True)
result_seedW.drop('TeamID', axis=1, inplace=True)

In [5]:
#Merge seed with losing team ids
result_seed = pd.merge(result_seedW, tourney_seed, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'])
result_seed.rename(columns={'Seed': 'LSeed'}, inplace=True)
result_seed.drop('TeamID', axis=1, inplace=True)

In [6]:
#Create function that removes the letter before the seed
def remove_region(region_seed):
    return int(region_seed[1:3])

In [7]:
#Apply the remove_region function to the dataframe
result_seed['WSeed'] = result_seed['WSeed'].apply(remove_region)
result_seed['LSeed'] = result_seed['LSeed'].apply(remove_region)

In [8]:
#Select the seasons that are consistent with the seasons with the efficiency stats dataframe
result_seed = result_seed[result_seed['Season'].isin([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])]
result_seed.reset_index(drop=True, inplace=True)
result_seed

,Season,WTeamID,LTeamID,WSeed,LSeed
0,2010,3124,3201,4,13
1,2010,3173,3395,8,9
2,2010,3181,3214,2,15
3,2010,3199,3256,3,14
4,2010,3207,3265,5,12
...,...,...,...,...,...
625,2019,3124,3234,1,2
626,2019,3323,3390,1,2
627,2019,3124,3332,1,2
628,2019,3323,3163,1,2


In [9]:
#Create copies of dataframes change the team ids from winning and losing to 1 and 2
rs_win = result_seed.copy()
rs_win.rename(columns={'WTeamID': 'TeamID_1', 'LTeamID': 'TeamID_2', 'WSeed': 'Seed_1', 'LSeed': 'Seed_2'}, inplace=True)
rs_lose = result_seed.copy()
rs_lose.rename(columns={'WTeamID': 'TeamID_2', 'LTeamID': 'TeamID_1', 'WSeed': 'Seed_2', 'LSeed': 'Seed_1'}, inplace=True)
rs_win['Result'] = 1
rs_lose['Result'] = 0

In [10]:
#Combine the winning and losing dataframes
rs_both = pd.concat((rs_win, rs_lose), sort=False).reset_index(drop=True)
rs_both

,Season,TeamID_1,TeamID_2,Seed_1,Seed_2,Result
0,2010,3124,3201,4,13,1
1,2010,3173,3395,8,9,1
2,2010,3181,3214,2,15,1
3,2010,3199,3256,3,14,1
4,2010,3207,3265,5,12,1
...,...,...,...,...,...,...
1255,2019,3234,3124,2,1,0
1256,2019,3390,3323,2,1,0
1257,2019,3332,3124,2,1,0
1258,2019,3163,3323,2,1,0


In [11]:
#Select the average pace, team's and opponent's average offensive, and defensive ratings
#and the team's winning percentage, opponent's winning percentage, opponent's opponent's winning percentage
reg_season = reg_season[['Season', 'TeamID', 'AvgPace', 'AvgOFF', 'AvgDEF', 'OAvgOFF', 'OAvgDEF', 'WinP', 'OWinP', 'OppOWinP', 'RPI']]
reg_group = reg_season.groupby(['Season', 'TeamID']).mean()
reg_group.reset_index(inplace=True)

In [12]:
#Merge regular season dataframe onto the result and seed dataframe by season and team id for both team ids
rts_1 = pd.merge(rs_both, reg_group, how='left', left_on=['Season', 'TeamID_1'], right_on=['Season', 'TeamID'])
rts_1.drop('TeamID', axis=1, inplace=True)
rts = pd.merge(rts_1, reg_group, how='left', left_on=['Season', 'TeamID_2'], right_on=['Season', 'TeamID'], suffixes=('_1', '_2'))
rts.drop(['TeamID_1', 'TeamID_2', 'TeamID', 'Season'], axis=1, inplace=True)
rts

,Seed_1,Seed_2,Result,AvgPace_1,AvgOFF_1,AvgDEF_1,OAvgOFF_1,OAvgDEF_1,WinP_1,OWinP_1,...,RPI_1,AvgPace_2,AvgOFF_2,AvgDEF_2,OAvgOFF_2,OAvgDEF_2,WinP_2,OWinP_2,OppOWinP_2,RPI_2
0,4,13,1,72.175781,99.658607,79.508594,95.616172,89.551714,0.718750,0.594413,...,0.616010,72.037879,101.686032,83.834909,90.513622,90.441033,0.818182,0.507732,0.511079,0.586181
1,8,9,1,72.897115,97.271769,81.753636,90.130587,87.869301,0.807692,0.536687,...,0.598059,73.391250,98.346002,81.973838,91.242103,90.611370,0.733333,0.523685,0.527571,0.577068
2,2,15,1,73.216797,97.418218,75.734771,92.736424,85.692251,0.843750,0.612990,...,0.657204,69.771667,90.506963,78.734554,85.620101,88.729290,0.633333,0.465572,0.468451,0.508232
3,3,14,1,73.925833,100.833748,81.679650,91.287937,86.795797,0.833333,0.545861,...,0.621337,76.128226,97.814863,84.530562,89.049872,90.049125,0.741935,0.478864,0.505811,0.551368
4,5,12,1,70.179583,97.228980,83.519539,90.336751,87.387895,0.800000,0.546405,...,0.612272,67.431061,100.934661,85.750907,88.719918,88.315877,0.787879,0.505540,0.489238,0.572049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,2,1,0,70.774609,113.157432,95.360543,98.300952,90.725940,0.812500,0.608047,...,0.643075,71.764453,111.740902,76.205442,97.748523,90.418103,0.968750,0.606965,0.557409,0.685022
1256,2,1,0,70.356250,106.042042,86.450779,101.249238,93.373623,0.875000,0.603816,...,0.661401,75.517803,117.857692,84.942546,100.128861,90.064887,0.909091,0.655450,0.571312,0.697826
1257,2,1,0,69.493750,122.527953,91.805827,99.927352,92.936460,0.875000,0.595451,...,0.657079,71.764453,111.740902,76.205442,97.748523,90.418103,0.968750,0.606965,0.557409,0.685022
1258,2,1,0,70.557955,116.680039,77.292557,93.450763,89.305359,0.939394,0.558694,...,0.651034,75.517803,117.857692,84.942546,100.128861,90.064887,0.909091,0.655450,0.571312,0.697826


In [13]:
#Separate results from rest of dataframe
X = rts.drop('Result', axis=1)
y = rts['Result']
print(X.shape, y.shape)

(1260, 20) (1260,)


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [15]:
#Transform previous seasons data using X_scaler
X_scaler = StandardScaler().fit(X)
X_scaled = X_scaler.transform(X)

In [16]:
#Set up 2021 submission predictions
pred_df = pd.read_csv('WSampleSubmissionStage2.csv')

#Separate game ids to season, and team 1 and 2 ids
pred_df['Season'] = pred_df['ID'].map(lambda x: int(x[:4]))
pred_df['TeamID_1'] = pred_df['ID'].map(lambda x: int(x[5:9]))
pred_df['TeamID_2'] = pred_df['ID'].map(lambda x: int(x[10:14]))

#Merge seeds onto 2021 matchups for both teams
pred_seedW = pd.merge(pred_df, tourney_seed, how='left', left_on=['Season', 'TeamID_1'], right_on=['Season', 'TeamID'])
pred_seedW.rename(columns={'Seed': 'Seed_1'}, inplace=True)
pred_seedW.drop('TeamID', axis=1, inplace=True)
pred_seed = pd.merge(pred_seedW, tourney_seed, how='left', left_on=['Season', 'TeamID_2'], right_on=['Season', 'TeamID'])
pred_seed.rename(columns={'Seed': 'Seed_2'}, inplace=True)
pred_seed.drop('TeamID', axis=1, inplace=True)

#Merge efficiency stats data onto 2021 data for both teams
pred_1 = pd.merge(pred_seed, reg_group, how='left', left_on=['Season', 'TeamID_1'], right_on=['Season', 'TeamID'])
pred_1.drop('TeamID', axis=1, inplace=True)
pred = pd.merge(pred_1, reg_group, how='left', left_on=['Season', 'TeamID_2'], right_on=['Season', 'TeamID'], suffixes=('_1', '_2'))
pred.drop(['TeamID_1', 'TeamID_2', 'TeamID', 'Season', 'ID', 'Pred'], axis=1, inplace=True)

#Removes the letter in front of the seeds
pred['Seed_1'] = pred['Seed_1'].apply(remove_region)
pred['Seed_2'] = pred['Seed_2'].apply(remove_region)
pred


,Seed_1,Seed_2,AvgPace_1,AvgOFF_1,AvgDEF_1,OAvgOFF_1,OAvgDEF_1,WinP_1,OWinP_1,OppOWinP_1,RPI_1,AvgPace_2,AvgOFF_2,AvgDEF_2,OAvgOFF_2,OAvgDEF_2,WinP_2,OWinP_2,OppOWinP_2,RPI_2
0,7,3,72.647000,103.316418,96.671827,94.578662,88.871802,0.640000,0.566643,0.556933,0.582555,71.278571,93.435681,78.439261,92.811835,90.732540,0.761905,0.519156,0.538624,0.584710
1,7,4,72.647000,103.316418,96.671827,94.578662,88.871802,0.640000,0.566643,0.556933,0.582555,77.562500,107.249602,95.879505,95.082642,88.985329,0.703704,0.553915,0.553273,0.591201
2,7,2,72.647000,103.316418,96.671827,94.578662,88.871802,0.640000,0.566643,0.556933,0.582555,76.075463,107.115988,71.984825,92.723836,92.784096,0.925926,0.519451,0.538943,0.625943
3,7,12,72.647000,103.316418,96.671827,94.578662,88.871802,0.640000,0.566643,0.556933,0.582555,68.208000,103.437506,85.500360,90.524237,92.457910,0.800000,0.470638,0.500253,0.560382
4,7,11,72.647000,103.316418,96.671827,94.578662,88.871802,0.640000,0.566643,0.556933,0.582555,70.789352,98.709404,92.415064,92.939867,91.324328,0.592593,0.530875,0.499023,0.538341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,9,13,70.092935,90.880800,90.910828,94.254972,89.602120,0.521739,0.557400,0.528272,0.541203,67.948500,93.636307,81.385710,89.650532,92.082917,0.720000,0.441153,0.494998,0.524326
2012,9,14,70.092935,90.880800,90.910828,94.254972,89.602120,0.521739,0.557400,0.528272,0.541203,66.426630,92.933162,86.371881,91.623844,92.169012,0.608696,0.493756,0.509227,0.526359
2013,4,13,73.962037,98.411410,89.347248,96.078204,91.003426,0.777778,0.546245,0.549050,0.604830,67.948500,93.636307,81.385710,89.650532,92.082917,0.720000,0.441153,0.494998,0.524326
2014,4,14,73.962037,98.411410,89.347248,96.078204,91.003426,0.777778,0.546245,0.549050,0.604830,66.426630,92.933162,86.371881,91.623844,92.169012,0.608696,0.493756,0.509227,0.526359


In [17]:
#Scale the 2021 submissions data
pred_scaled = X_scaler.transform(pred)

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [19]:
#Create an ensemble of 30 models to reduce variance of predictions

for i in range(1,31):  
    #Create model with a layer of 200 hidden nodes with 20 inputs
    model = Sequential()
    model.add(Dense(units=200, activation='relu', input_dim=20))
    model.add(Dense(units=1, activation='sigmoid'))

    #Compile model optimizing for log loss
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    #Fit model
    model.fit(
        X_scaled,
        y,
        epochs=100,
        shuffle=True,
        verbose=0
    )

    #Use model to predict probabilities of 2021 games
    predictions = model.predict_proba(pred_scaled)

    #Reload the submission csv to get format
    submission_df = pd.read_csv('WSampleSubmissionStage2.csv')

    #Overwrite Pred column with model's predictions
    submission_df['Pred'] = predictions
    
    if i == 1:
        models_all = submission_df
        submission_df.to_csv('check1.csv', index=False)
    else:
        models_all = models_all.merge(submission_df, on='ID')

In [20]:
#Average predictions of all models
models_all['Pred'] = models_all.mean(axis=1)

#Select needed columns and write to csv
ens_model = models_all[['ID', 'Pred']]
ens_model.to_csv('predictions.csv', index=False)